In [ ]:
import json
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from split_query.core import object_hook, And
from split_query.extract import extract_parameters
from split_query.core.expand import get_clauses

In [ ]:
# Simplification timings
results = pd.read_csv('benchmarks.csv')
grouped = pd.DataFrame(results).drop(columns=['ind']).groupby(['name', 'clauses']).mean()
grouped = grouped.apply(np.log)
grouped = grouped - grouped.min().min()
_, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(14, 6))
grouped.loc['in-memory'].plot(ax=ax1, kind='bar')
grouped.loc['persistent'].plot(ax=ax2, kind='bar');

In [ ]:
# Execution path through the cache
with open('path_inmemory.json') as infile:
    path_inmemory = json.load(infile, object_hook=object_hook)
with open('path_persistent.json') as infile:
    path_persistent = json.load(infile, object_hook=object_hook)

parameters = [
    dict(attr='sensor', type='tag', key='sensor', single=True),
    dict(attr='datetime', type='range', key_lower='lower', key_upper='upper')]

def extract(expression):
    res = extract_parameters(expression, parameters)
    assert len(res) == 1
    return res[0][1]

def print_path(path):
    for _, current, cached in path:
        keys = extract(cached)
        print('{:<25}{:<20}{:>3}'.format(
            str(keys['lower']), keys['sensor'],
            len(get_clauses(And([current, cached])))))

print('in-memory')
print_path(path_inmemory)
print('\npersistent')
print_path(path_persistent)